Set-up environment via terminal:
- Navigate to `home/bedmap` (local repository location on Roger)
- `conda list -e` to show available environments
- `conda activate /home/kim/bedmap/.conda-bedmap` (Python 3.8.18)

Then just use the following commands as normal:
- `conda install x` to install new packages
    - e.g. `conda install python=3 geopandas` needs to be python 3, and conda-forge needs to be added 
- `conda list` to see all installed packages

## Install packages

In [26]:
import pandas as pd
# Geopandas is a geo dataframe
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

# Check metadata

In [30]:
# This example is BM2
path_to_bas_agap = "/home/kim/data/bedmap/BAS_2007_AGAP_AIR_BM2.csv"
# LDEO Rosetta of the full Ross ice sheet
path_to_rosetta = "/home/kim/data/bedmap/bedmap3-csv/LDEO_2015_ROSETTA_AIR_BM3.csv"

In [31]:
# seperate is not a comma but a colon with a space
bas_agap_meta = pd.read_csv(path_to_rosetta, nrows = 18, sep = ": ", engine = "python", header = None)
bas_agap_meta

,0,1
0,#project,Ross Ice Shelf (ROSETTA).
1,#time_coverage_start,2015
2,#time_coverage_end,2017
3,#creator_name,R. Bell (Lamont-Doherty Earth Observatory).
4,#institution,Lamont-Doherty Earth Observatory.
5,#acknowledgement,NSF grant 0958658; 1443543; 1443677; 1443497; ...
6,#source,http://www.ldeo.columbia.edu/polar-geophysics-...
7,#references,https://doi.org/10.1029/2019JF005241
8,#platform,airborne radar.
9,#instrument,DICE IcePod.


# Load data

- skip meta data
- Rosetta has 2.65 M datapoints

In [32]:
# Load data  into a pandas dataframe
# skip 18 rows of metadata
pd_data = pd.read_csv(path_to_rosetta, skiprows = 18, low_memory = False)
pd_data.tail()

,trajectory_id,trace_number,longitude (degree_east),latitude (degree_north),date,time_UTC,surface_altitude (m),land_ice_thickness (m),bedrock_altitude (m),two_way_travel_time (m),aircraft_altitude (m),along_track_distance (m)
2651548,T1110,-9999,173.014007,-77.429764,2017-11-29,20:47:34.86,-440.545754,44.582638,-9999,-9999,725.481529,-9999
2651549,T1110,-9999,173.014100,-77.429601,2017-11-29,20:47:35.03,-287.678752,81.096675,-9999,-9999,725.476059,-9999
2651550,T1110,-9999,173.014193,-77.429437,2017-11-29,20:47:35.19,-177.775326,118.181577,-9999,-9999,725.471172,-9999
2651551,T1110,-9999,173.014286,-77.429273,2017-11-29,20:47:35.35,-20.910828,125.722537,-9999,-9999,725.466533,-9999
2651552,T1110,-9999,173.014380,-77.429109,2017-11-29,20:47:35.52,-339.649337,101.756597,-9999,-9999,725.462131,-9999


In [33]:
# Convert pandas dataframe to xarray data set
xr_data = xr.Dataset.from_dataframe(pd_data)
xr_data

<xarray.Dataset>
Dimensions:                   (index: 2651553)
Coordinates:
  * index                     (index) int64 0 1 2 3 ... 2651550 2651551 2651552
Data variables:
    trajectory_id             (index) object 'L190' 'L190' ... 'T1110' 'T1110'
    trace_number              (index) int64 -9999 -9999 -9999 ... -9999 -9999
    longitude (degree_east)   (index) float64 -158.3 -158.3 ... 173.0 173.0
    latitude (degree_north)   (index) float64 -83.76 -83.76 ... -77.43 -77.43
    date                      (index) object '2015-12-01' ... '2017-11-29'
    time_UTC                  (index) object '16:40:17.35' ... '20:47:35.52'
    surface_altitude (m)      (index) float64 60.3 60.33 60.36 ... -20.91 -339.6
    land_ice_thickness (m)    (index) float64 645.2 645.4 645.5 ... 125.7 101.8
    bedrock_altitude (m)      (index) int64 -9999 -9999 -9999 ... -9999 -9999
    two_way_travel_time (m)   (index) int64 -9999 -9999 -9999 ... -9999 -9999
    aircraft_altitude (m)     (index) float64 801.7 801.7 801.8 ... 725.5 725.5
    along_track_distance (m)  (index) int64 -9999 -9999 -9999 ... -9999 -9999

# Convert to Polar Stereographic

In [ ]:
lonlat_to_polarstereo = pyproj.Transformer.from_crs(crs_from = pyproj.CRS("epsg:4326"),  
                                                    crs_to = pyproj.CRS("epsg:3031"),
                                                    always_xy = True) #lonlat

# Geotif

In [8]:
# Create a geo dataframe
gdf = gpd.GeoDataFrame(data, 
                       geometry = gpd.points_from_xy(data['longitude (degree_east)'], data['latitude (degree_north)']))
 

In [10]:
gdf.head()

,trajectory_id,trace_number,longitude (degree_east),latitude (degree_north),date,time_UTC,surface_altitude (m),land_ice_thickness (m),bedrock_altitude (m),two_way_travel_time (m),aircraft_altitude (m),along_track_distance (m),geometry
0,3,-9999,169.169538,-77.812387,2008-12-06,04:02:51.30,-21.97,-9999.0,-9999.0,-9999,-9999,-9999,POINT (169.16954 -77.81239)
1,3,-9999,169.170490,-77.812373,2008-12-06,04:02:51.63,-21.78,-9999.0,-9999.0,-9999,-9999,-9999,POINT (169.17049 -77.81237)
2,3,-9999,169.171442,-77.812358,2008-12-06,04:02:51.97,-21.56,-9999.0,-9999.0,-9999,-9999,-9999,POINT (169.17144 -77.81236)
3,3,-9999,169.172393,-77.812342,2008-12-06,04:02:52.30,-21.40,-9999.0,-9999.0,-9999,-9999,-9999,POINT (169.17239 -77.81234)
4,3,-9999,169.173341,-77.812326,2008-12-06,04:02:52.63,-21.24,-9999.0,-9999.0,-9999,-9999,-9999,POINT (169.17334 -77.81233)


In [11]:
gdf = gdf.set_crs("EPSG:4326")

In [12]:
# Convert geography column to Polar Stereographic
gdf = gdf.to_crs("EPSG:3031")
# https://epsg.io/3031

In [14]:
gdf.head()

,trajectory_id,trace_number,longitude (degree_east),latitude (degree_north),date,time_UTC,surface_altitude (m),land_ice_thickness (m),bedrock_altitude (m),two_way_travel_time (m),aircraft_altitude (m),along_track_distance (m),geometry
0,3,-9999,169.169538,-77.812387,2008-12-06,04:02:51.30,-21.97,-9999.0,-9999.0,-9999,-9999,-9999,POINT (249729.026 -1305354.514)
1,3,-9999,169.170490,-77.812373,2008-12-06,04:02:51.63,-21.78,-9999.0,-9999.0,-9999,-9999,-9999,POINT (249707.626 -1305360.173)
2,3,-9999,169.171442,-77.812358,2008-12-06,04:02:51.97,-21.56,-9999.0,-9999.0,-9999,-9999,-9999,POINT (249686.246 -1305365.940)
3,3,-9999,169.172393,-77.812342,2008-12-06,04:02:52.30,-21.40,-9999.0,-9999.0,-9999,-9999,-9999,POINT (249664.910 -1305371.811)
4,3,-9999,169.173341,-77.812326,2008-12-06,04:02:52.63,-21.24,-9999.0,-9999.0,-9999,-9999,-9999,POINT (249643.641 -1305377.667)


In [17]:
plt.rcParams["figure.figsize"] = (100, 100)
fig, ax = plt.subplots(1, 1)
fig = plt.figure(figsize = (100, 100), dpi = 100, facecolor = 'w', edgecolor = 'k')
gdf.plot(column = 'land_ice_thickness (m)', ax = ax, legend = True, legend_kwds = {'label': "Ice Thickness (m)",'orientation': "horizontal"})

<Axes: >

<Figure size 10000x10000 with 0 Axes>

In [19]:
# Export formats in geopandas
gdf.to_file("/home/kim/data/bedmap/geopackage/BAS_2007_AGAP_AIR_BM2_points.gpkg", layer = 'Points', driver = "GPKG")


In [20]:
gdf.to_file("/home/kim/data/bedmap/shapefile/BAS_2007_AGAP_AIR_BM2_points.gpkg", driver = "ESRI Shapefile")

/tmp/ipykernel_5732/1459969447.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("/home/kim/data/bedmap/shapefile/BAS_2007_AGAP_AIR_BM2_points.gpkg", driver = "ESRI Shapefile")


Additional steps do not work on top of the already provided files but start from scratch with the .csv files
- refIm = "input_grid/BEDMAP3_shapefile_processing_Input_extent500x500_crop.tif" not provided

In [21]:
path_to_raster = "/home/kim/data/bedmap/input_grid/BEDMAP3_shapefile_processing_Input_extent500x500_crop.tif"

Use gdal library for raster

https://gdal.org/index.html 
Gdal install issues: Install wheeln straight from here: http://pypi.naturalcapitalproject.org/simple/gdal/index.html
and install locally from wheel
cp is the python version 
gdal only supported up to python 3.08 

"/usr/bin/python3.8"

virtualenv --python="/usr/bin/python3.8"
venv --python="/usr/bin/python3.8"

virtualenv .venv --python="/usr/bin/python3.8"

assign indices to cells and then 

In [ ]:
ds = gdal.Open(path_to_raster)

In [ ]:
# Opening reference GeoTIFF and get relevant parameters
refIm = "input_grid/BEDMAP3_shapefile_processing_Input_extent500x500_crop.tif" 
ds = gdal.Open(refIm)
ar = ds.GetRasterBand(1).ReadAsArray()
gt = ds.GetGeoTransform()
res = gt[1]
xmin = gt[0]
ymax = gt[3]
xsize = ds.RasterXSize
ysize = ds.RasterYSize
ds = None
xstart = xmin + res/2 
ystart = ymax - res/2

# Create array of x and y values at the centre of each pixel
x = np.arange(xstart, xstart + xsize * res, res)
y = np.arange(ystart, ystart -ysize * res, -res )